In [ ]:
import pypsa
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import seaborn as sns

import sys
import os

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
from matplotlib.patches import Circle, Patch
from matplotlib.legend_handler import HandlerPatch


import cartopy.crs as ccrs
import cartopy

import multiprocessing as mp
from itertools import product

from pypsa.plot import projected_area_factor
from pypsa.descriptors import get_switchable_as_dense as as_dense

PATH = "../workflows"
SCRIPTS_PATH = "pypsa-eur-sec/scripts/"

sys.path.append(os.path.join(PATH, SCRIPTS_PATH))
from plot_summary import rename_techs
from plot_network import assign_location
from helper import override_component_attrs

import holoviews as hv
from holoviews import opts, dim

hv.extension("bokeh")
hv.output(size=200)

plt.style.use(["bmh", "matplotlibrc"])
xr.set_options(display_style="html")

%matplotlib inline

In [ ]:
CLUSTERS = 181
LL = "1.0"
OPTS = "Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-onwind+p0"
RUN = "20221227-main"
SCENARIO = f"elec_s_{CLUSTERS}_lv{LL}__{OPTS}_2050"
OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

OUTPUT = "../results/graphics-20221227/"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/{SCENARIO}/"

In [ ]:
if not os.path.exists(OUTPUT_SCENARIO):
    os.makedirs(OUTPUT_SCENARIO)

In [ ]:
with open(f"{PATH}/pypsa-eur-sec/config.main.yaml") as file:
    config = yaml.safe_load(file)

In [ ]:
tech_colors = config["plotting"]["tech_colors"]

In [ ]:
fn = f"{PATH}/pypsa-eur/resources/regions_onshore_elec_s_{CLUSTERS}.geojson"
nodes = gpd.read_file(fn).set_index("name")

fn = f"{PATH}/pypsa-eur/resources/regions_offshore_elec_s_{CLUSTERS}.geojson"
offnodes = gpd.read_file(fn).set_index("name")

fn = f"{PATH}/pypsa-eur/resources/country_shapes.geojson"
cts = gpd.read_file(fn).set_index("name")

regions = pd.concat(
    [
        gpd.read_file(f"{PATH}/pypsa-eur/resources/regions_onshore.geojson"),
        gpd.read_file(f"{PATH}/pypsa-eur/resources/regions_offshore.geojson"),
    ]
)
regions = regions.dissolve("name")

fn = f"{PATH}/pypsa-eur/resources/regions_onshore.geojson"
onregions = gpd.read_file(fn).set_index("name")

fn = f"{PATH}/pypsa-eur/resources/regions_onshore.geojson"
offregions = gpd.read_file(fn).set_index("name")

epsg = 3035
regions["Area"] = regions.to_crs(epsg=epsg).area.div(1e6)
onregions["Area"] = onregions.to_crs(epsg=epsg).area.div(1e6)
offregions["Area"] = offregions.to_crs(epsg=epsg).area.div(1e6)
nodes["Area"] = nodes.to_crs(epsg=epsg).area.div(1e6)

In [ ]:
europe_shape = nodes.dissolve()
europe_shape.index = ["EU"]

In [ ]:
minx, miny, maxx, maxy = europe_shape.explode(ignore_index=True).total_bounds
BOUNDARIES = [minx, maxx - 4, miny, maxy]

In [ ]:
overrides = override_component_attrs(OVERRIDES)
fn = f"{PATH}/pypsa-eur-sec/results/{RUN}/postnetworks/{SCENARIO}.nc"
n = pypsa.Network(fn, override_component_attrs=overrides)

In [ ]:
unique_link_carriers = n.links.carrier.unique()
GAS_NETWORK = "gas pipeline" in unique_link_carriers
H2_NETWORK = any("H2 pipeline" in ulc for ulc in unique_link_carriers)

## Utilities

In [ ]:
class HandlerCircle(HandlerPatch):
    """
    Legend Handler used to create circles for legend entries.

    This handler resizes the circles in order to match the same dimensional
    scaling as in the applied axis.
    """

    def create_artists(
        self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans
    ):
        fig = legend.get_figure()
        ax = legend.axes

        unit = np.diff(ax.transData.transform([(0, 0), (1, 1)]), axis=0)[0][1]
        radius = orig_handle.get_radius() * unit * (72 / fig.dpi)
        center = 5 - xdescent, 3 - ydescent
        p = plt.Circle(center, radius)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]


def add_legend_circles(
    ax, sizes, labels, scale=1, srid=None, patch_kw={}, legend_kw={}
):
    if srid is not None:
        area_correction = projected_area_factor(ax, n.srid) ** 2
        sizes = [s * area_correction for s in sizes]

    handles = make_legend_circles_for(sizes, scale, **patch_kw)

    legend = ax.legend(
        handles, labels, handler_map={Circle: HandlerCircle()}, **legend_kw
    )

    ax.add_artist(legend)


def add_legend_lines(ax, sizes, labels, scale=1, patch_kw={}, legend_kw={}):
    handles = [Line2D([0], [0], linewidth=s / scale, **patch_kw) for s in sizes]

    legend = ax.legend(handles, labels, **legend_kw)

    ax.add_artist(legend)


def add_legend_patch(ax, colors, labels, patch_kw={}, legend_kw={}):
    handles = [Patch(facecolor=c, **patch_kw) for c in colors]

    legend = ax.legend(handles, labels, **legend_kw)

    ax.add_artist(legend)


def make_legend_circles_for(sizes, scale=1.0, **kw):
    return [Circle((0, 0), radius=(s / scale) ** 0.5, **kw) for s in sizes]

In [ ]:
def nodal_balance(n, carrier, time=slice(None), aggregate=None, energy=True):
    if not isinstance(carrier, list):
        carrier = [carrier]

    one_port_data = {}

    for c in n.iterate_components(n.one_port_components):
        df = c.df[c.df.bus.map(n.buses.carrier).isin(carrier)]

        if df.empty:
            continue

        s = c.pnl.p.loc[time, df.index] * df.sign

        s = s.groupby([df.bus.map(n.buses.location), df.carrier], axis=1).sum()

        one_port_data[c.list_name] = s

    branch_data = {}

    for c in n.iterate_components(n.branch_components):
        for col in c.df.columns[c.df.columns.str.startswith("bus")]:
            end = col[3:]

            df = c.df[c.df[col].map(n.buses.carrier).isin(carrier)]

            if df.empty:
                continue

            s = -c.pnl[f"p{end}"].loc[time, df.index]

            s = s.groupby([df[col].map(n.buses.location), df.carrier], axis=1).sum()

            branch_data[(c.list_name, end)] = s

    branch_balance = pd.concat(branch_data).groupby(level=[0, 2]).sum()
    one_port_balance = pd.concat(one_port_data)

    def skip_tiny(df, threshold=1e-1):
        return df.where(df.abs() > threshold)

    branch_balance = skip_tiny(branch_balance)
    one_port_balance = skip_tiny(one_port_balance)

    balance = pd.concat([one_port_balance, branch_balance]).stack(level=[0, 1])

    balance.index.set_names(["component", "bus"], level=[0, 2], inplace=True)

    if energy:
        balance = balance * n.snapshot_weightings.generators

    if aggregate is not None:
        keep_levels = balance.index.names.difference(aggregate)
        balance = balance.groupby(level=keep_levels).sum()

    return balance

In [ ]:
def get_import_export_balance(n, with_methanol=False):
    io = pd.DataFrame()

    io["oil"] = get_import_export_oil(with_methanol).eval("demand - supply")

    balance = nodal_balance(n, "AC", aggregate=["snapshot"])
    h2_balance = nodal_balance(n, "H2", aggregate=["snapshot"])
    io["hydrogen"] = (
        h2_balance.droplevel(0)
        .unstack("carrier")
        .filter(like="H2 pipeline")
        .sum(axis=1)
    ).div(1e6)

    ac_balance = nodal_balance(n, "AC", aggregate=["snapshot"])
    io["electricity"] = (
        ac_balance.droplevel(0).unstack("carrier")[["AC", "DC"]].sum(axis=1).div(1e6)
    )

    if GAS_NETWORK:
        gas_balance = nodal_balance(n, "gas", aggregate=["snapshot"])

        io["gas"] = (
            gas_balance.groupby(["bus", "carrier"])
            .sum()
            .unstack()
            .filter(like="gas pipeline")
            .sum(axis=1)
            .div(1e6)
        )

    else:
        io["gas"] = get_import_export_gas().eval("demand - supply")

    io["total"] = io.sum(axis=1)

    return io.fillna(0.0)

In [ ]:
def get_import_export_gas():
    io = pd.DataFrame()

    fn = f"{PATH}/pypsa-eur-sec/resources/industrial_energy_demand_elec_s_{CLUSTERS}_2050.csv"
    industrial_demand = pd.read_csv(fn, index_col=0)

    fn = f"{PATH}/pypsa-eur-sec/resources/biomass_potentials_s_{CLUSTERS}.csv"
    biogas_potentials = pd.read_csv(fn, index_col=0)["biogas"].div(1e6)  # TWh

    e = n.stores_t.e.loc[:, "EU biogas"]
    biogas_used = (e[0] - e[-1]) / e[0]

    biogas_generation = biogas_potentials * biogas_used

    gas_cc_consumption = n.snapshot_weightings.generators @ (
        n.links_t.p0.loc[:, "gas for industry CC"]
        + n.links_t.p1.loc[:, "gas for industry CC"]
    ).div(1e6)

    gas_consumption = n.snapshot_weightings.generators @ n.links_t.p0.filter(
        regex="(OCGT|gas (boiler|CHP)|SMR)", axis=1
    )
    grouper = gas_consumption.index.map(n.links.bus1.map(n.buses.location))
    io["demand"] = (
        gas_consumption.groupby(grouper).sum().div(1e6) + industrial_demand["methane"]
    )  # TWh

    gas_generation = n.snapshot_weightings.generators @ n.links_t.p1.filter(
        regex="(Sabatier|helmeth)", axis=1
    ).mul(-1)
    grouper = gas_generation.index.map(n.links.bus0.map(n.buses.location))
    io["supply"] = (
        gas_generation.groupby(grouper).sum().div(1e6) + biogas_generation
    )  # TWh, assuming biogas potentials uniformly exhausted

    fossil_gas = n.snapshot_weightings.generators @ n.generators_t.p.loc[
        :, "EU gas"
    ].div(1e6)
    io.at["fossil", "supply"] = fossil_gas

    assert abs(io["supply"].sum() - gas_cc_consumption - io["demand"].sum()) < 1

    return io.fillna(0.0)

In [ ]:
def get_import_export_oil(with_methanol=False):
    io = pd.DataFrame()

    fn = f"{PATH}/pypsa-eur-sec/resources/industrial_energy_demand_elec_s_{CLUSTERS}_2050.csv"
    industrial_demand = pd.read_csv(fn, index_col=0)

    fn = f"{PATH}/pypsa-eur-sec/resources/pop_layout_elec_s_{CLUSTERS}.csv"
    pop_layout = pd.read_csv(fn, index_col=0)

    fn = f"{PATH}/pypsa-eur-sec/resources/pop_weighted_energy_totals_s_{CLUSTERS}.csv"
    nodal_energy_totals = pd.read_csv(fn, index_col=0)

    oil = [
        "total international aviation",
        "total domestic aviation",
        "total agriculture machinery",
    ]
    if with_methanol:
        oil.append("total domestic navigation")

    io["demand"] = industrial_demand["naphtha"] + nodal_energy_totals[oil].sum(axis=1)

    if with_methanol:
        fn = f"{PATH}/pypsa-eur-sec/resources/shipping_demand_s_{CLUSTERS}.csv"
        efficiency = (
            config["sector"]["shipping_oil_efficiency"]
            / config["sector"]["shipping_methanol_efficiency"]
        )
        io["demand"] += pd.read_csv(fn, index_col=0).squeeze() * efficiency
        regex = "(Fischer-Tropsch|methanolisation)"
    else:
        regex = "Fischer-Tropsch"

    oil_generation = n.snapshot_weightings.generators @ n.links_t.p1.filter(
        regex=regex, axis=1
    ).mul(-1)
    grouper = oil_generation.index.map(n.links.bus0.map(n.buses.location))
    io["supply"] = oil_generation.groupby(grouper).sum().div(1e6)  # TWh

    fossil_oil = n.snapshot_weightings.generators @ n.generators_t.p.loc[
        :, "EU oil"
    ].div(1e6)
    io.at["fossil", "supply"] = fossil_oil

    # assert abs(io["supply"].sum() - io["demand"].sum()) < 1

    return io.fillna(0.0)

## Nodal Balance CSV Exports

In [ ]:
carriers = [
    "AC",
    "co2",
    "co2 stored",
    "gas",
    "H2",
    "oil",
    "methanol",
]

for carrier in carriers:
    print(carrier)
    df = (
        nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
        .unstack()
        .fillna(0.0)
        .div(1e3)
        .round(1)
    )
    df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-{carrier}.csv")

In [ ]:
carrier = ["AC", "low voltage"]
df = (
    nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
    .unstack()
    .fillna(0.0)
    .div(1e3)
    .round(1)
)
df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-total-electricity.csv")

In [ ]:
carrier = [
    "residential rural heat",
    "services rural heat",
    "residential urban decentral heat",
    "services urban decentral heat",
    "urban central heat",
]
df = (
    nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
    .unstack()
    .fillna(0.0)
    .div(1e3)
    .round(1)
)
df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-total-heat.csv")

## Liquid Hydrocarbon Demand

In [ ]:
n.storage_units.eval("capital_cost * p_nom_opt").div(1e9).groupby(
    n.storage_units.carrier
).sum()

In [ ]:
n.generators.eval("capital_cost * p_nom_opt").div(1e9).groupby(
    n.generators.carrier
).sum()

## Curtailment

In [ ]:
available = n.snapshot_weightings.generators @ (
    n.generators_t.p_max_pu * n.generators.p_nom_opt
)

In [ ]:
used = n.snapshot_weightings.generators @ n.generators_t.p

In [ ]:
available = (
    available.groupby(n.generators.carrier).sum().drop(["gas", "oil"]).div(1e6)
)  # TWh

In [ ]:
used = used.groupby(n.generators.carrier).sum().drop(["gas", "oil"]).div(1e6)  # TWh

In [ ]:
curtailed = available - used

In [ ]:
pd.concat(
    {"curtailed": curtailed, "available": available, "used": used}, axis=1
).to_csv(f"{OUTPUT_SCENARIO}/curtailment.csv")

## Storage SOC

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(10, 12))

ax = axs[0]
n.stores_t.e.filter(like="H2 Store", axis=1).sum(axis=1).div(1e6).plot(
    ax=ax,
    ylabel="SOC [TWh]",
    xlabel="",
    color=tech_colors["hydrogen storage"],
    ylim=(0, 65),
)
ax.set_title("Hydrogen Storage (full year)", fontsize=11)

ax = axs[2]
n.stores_t.e.filter(like="water tank", axis=1).sum(axis=1).div(1e6).plot(
    ax=ax,
    ylabel="SOC [TWh]",
    xlabel="",
    color=tech_colors["hot water storage"],
    ylim=(0, 65),
)
ax.set_title("Hot Water Storage (full year)", fontsize=11)

ax = axs[1]
n.stores_t.e.filter(like="H2 Store", axis=1).sum(axis=1).loc["2013-07":"2013-08"].div(
    1e6
).plot(ax=ax, ylabel="SOC [TWh]", xlabel="", color=tech_colors["hydrogen storage"])
ax.set_title("Hydrogen Storage (July/August)", fontsize=11)

ax = axs[3]
n.stores_t.e.filter(like="water tank", axis=1).sum(axis=1).loc["2013-07":"2013-08"].div(
    1e6
).plot(ax=ax, ylabel="SOC [TWh]", xlabel="", color=tech_colors["hot water storage"])
ax.set_title("Hot Water Storage (July/August)", fontsize=11)

ax = axs[4]
n.stores_t.e.filter(like="battery", axis=1).sum(axis=1).loc["2013-07":"2013-08"].div(
    1e6
).plot(ax=ax, ylabel="SOC [TWh]", xlabel="", color=tech_colors["battery"], ylim=(0, 8))
ax.set_title("Battery Storage (July/August)", fontsize=11)

plt.tight_layout()

plt.savefig(f"{OUTPUT_SCENARIO}/soc.pdf", bbox_inches="tight")

## Gini, Lorenz, Equity

In [ ]:
def cumulative_share(n, carrier, sortby="supply/demand", df=None):
    if df is None:
        to_drop = [
            "AC",
            "DC",
            "H2 pipeline",
            "gas pipeline",
            "gas pipeline new",
            "H2 pipeline retrofitted",
        ]

        balance = nodal_balance(n, carrier, aggregate=["snapshot"])

        balance = balance.groupby(["bus", "carrier"]).sum().unstack("carrier")
        balance.drop(balance.columns.intersection(to_drop), axis=1, inplace=True)

        supply = balance.where(balance > 0).sum(axis=1)
        demand = -balance.where(balance < 0).sum(axis=1)

        df = pd.concat({"supply": supply, "demand": demand}, axis=1)

    df = df.div(df.sum()) * 100

    df["sortby"] = df.eval(sortby)

    df.sort_values(by="sortby", inplace=True)

    df.drop("sortby", axis=1, inplace=True)

    lead = pd.DataFrame({"supply": [0], "demand": [0]})

    df = pd.concat([lead, df])

    return df.cumsum()

In [ ]:
def plot_lorenz(n, carriers, sortby="supply/demand", dfs={}, fn=None):
    for c in set(carriers).difference(dfs.keys()):
        dfs[c] = None

    fig, ax = plt.subplots(figsize=(3.5, 3.5))

    linspace = np.linspace(0, 100, 100)
    ax.plot(linspace, linspace, c="darkgrey", linestyle=":", linewidth=1)

    nice_names = {
        "AC": "electricity",
        "H2": "hydrogen",
        "gas": "methane",
        "oil": "oil",
    }

    for carrier in carriers:
        df = cumulative_share(n, carrier, sortby, df=dfs[carrier])
        ax.plot(
            df.demand,
            df.supply,
            label=nice_names[carrier],
            color=tech_colors[nice_names[carrier]],
        )

    ax.set_xlabel(f"Cumulative Share of Demand [%]")
    ax.set_ylabel(f"Cumulative Share of Supply [%]")
    ax.set_ylim(-1, 101)
    ax.set_xlim(-1, 101)
    ax.legend()

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
carriers = ["oil", "gas", "H2", "AC"]

dfs = {"oil": get_import_export_oil()}
if not GAS_NETWORK:
    dfs["gas"] = get_import_export_gas()

plot_lorenz(
    n,
    carriers=carriers,
    sortby="supply/demand",
    dfs=dfs,
    fn=f"{OUTPUT_SCENARIO}/lorenz.pdf",
)

## Nodal price duration curve

In [ ]:
def plot_price_duration_curve(n, carrier, fn=None):
    df = n.buses_t.marginal_price.loc[:, n.buses.carrier == carrier]

    df = df.stack()

    to_plot = df.sort_values(ascending=False).reset_index(drop=True)

    to_plot.index = [i / len(df) * 100 for i in to_plot.index]

    fig, ax = plt.subplots(figsize=(4, 2.5))
    to_plot.plot(
        ax=ax,
    )
    plt.xlabel("share of snapshots and nodes [%]")
    plt.ylabel("nodal price [EUR/MWh]")
    plt.axvline(0, linewidth=0.5, linestyle=":", color="grey")
    plt.axvline(100, linewidth=0.5, linestyle=":", color="grey")
    plt.axhline(0, linewidth=0.5, linestyle=":", color="grey")

    if carrier == "H2":
        title = "Hydrogen"
        plt.ylim([-20, 350])
    elif carrier == "AC":
        title = "Electricity"
        plt.ylim([-100, 1000])
    else:
        title = carrier

    plt.title(title, fontsize=12, color="#343434")

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
carriers = [
    "AC",
    "H2",
    "residential rural heat",
    "services rural heat",
    "residential urban decentral heat",
    "services urban decentral heat",
    "urban central heat",
    "low voltage",
]

if GAS_NETWORK:
    carriers.append("gas")

In [ ]:
for carrier in carriers:
    plot_price_duration_curve(
        n, carrier, fn=f"{OUTPUT_SCENARIO}/price-duration-{carrier}.pdf"
    )

## Maps: Flow Patterns

In [ ]:
def plot_h2_flow(
    network, regions=None, series=None, min_energy=2e6, lim=100, fn=None
):  # MWh net transported
    crs = ccrs.EqualEarth()

    n = network.copy()

    assign_location(n)

    linewidth_factor = 8e5
    # MW below which not drawn
    line_lower_threshold = 1e2
    link_color = "k"

    # Drop non-electric buses so they don't clutter the plot
    n.buses.drop(n.buses.index[n.buses.carrier != "AC"], inplace=True)

    n.mremove("Link", n.links[~n.links.carrier.str.contains("H2 pipeline")].index)

    n.links.bus0 = n.links.bus0.str.replace(" H2", "")
    n.links.bus1 = n.links.bus1.str.replace(" H2", "")

    n.links["flow"] = n.snapshot_weightings.generators @ n.links_t.p0

    positive_order = n.links.bus0 < n.links.bus1
    swap_buses = {"bus0": "bus1", "bus1": "bus0"}
    n.links.loc[~positive_order] = n.links.rename(columns=swap_buses)
    n.links.loc[~positive_order, "flow"] = -n.links.loc[~positive_order, "flow"]
    n.links.index = n.links.apply(lambda x: f"H2 pipeline {x.bus0} -> {x.bus1}", axis=1)
    n.links = n.links.groupby(n.links.index).agg(
        dict(flow="sum", bus0="first", bus1="first", carrier="first", p_nom_opt="sum")
    )

    n.links.flow = n.links.flow.where(n.links.flow.abs() > min_energy)

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    n.plot(
        bus_sizes=0,
        link_colors=link_color,
        branch_components=["Link"],
        ax=ax,
        geomap=True,
        flow=pd.concat({"Link": n.links.flow.div(linewidth_factor)}),
    )

    if regions is not None:
        regions = regions.to_crs(crs.proj4_init)

        regions.plot(
            ax=ax,
            column=-series.reindex(regions.index),
            cmap="PRGn",
            linewidths=0,
            legend=True,
            vmax=lim,
            vmin=-lim,
            legend_kwds={
                "label": "Hydrogen balance [TWh]",
                "shrink": 0.7,
                "extend": "both",
            },
        )

    colors = [link_color]
    labels = ["Hydrogen flows [TWh]"]
    add_legend_patch(
        ax,
        colors,
        labels,
        patch_kw=dict(edgecolor="k"),
        legend_kw=dict(loc="upper left"),
    )

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
def plot_gas_flow(
    network,
    min_energy=2e6,  # MWh net transported
    max_energy=100e6,
    fn=None,
):
    crs = ccrs.EqualEarth()

    n = network.copy()

    assign_location(n)

    linewidth_factor = 15e5
    # MW below which not drawn
    line_lower_threshold = 1e2
    link_color = "indianred"

    # Drop non-electric buses so they don't clutter the plot
    n.buses.drop(n.buses.index[n.buses.carrier != "AC"], inplace=True)

    n.mremove("Link", n.links[~n.links.carrier.str.contains("gas pipeline")].index)

    n.links.bus0 = n.links.bus0.str.replace(" gas", "")
    n.links.bus1 = n.links.bus1.str.replace(" gas", "")

    n.links["flow"] = n.snapshot_weightings.generators @ n.links_t.p0

    positive_order = n.links.bus0 < n.links.bus1
    swap_buses = {"bus0": "bus1", "bus1": "bus0"}
    n.links.loc[~positive_order] = n.links.rename(columns=swap_buses)
    n.links.loc[~positive_order, "flow"] = -n.links.loc[~positive_order, "flow"]
    n.links.index = n.links.apply(
        lambda x: f"gas pipeline {x.bus0} -> {x.bus1}", axis=1
    )
    n.links = n.links.groupby(n.links.index).agg(
        dict(flow="sum", bus0="first", bus1="first", carrier="first", p_nom_opt="sum")
    )

    n.links.flow = n.links.flow.where(n.links.flow.abs() > min_energy)

    n.links.flow = n.links.flow.clip(upper=max_energy)

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    n.plot(
        bus_sizes=0,
        link_colors=link_color,
        projection=crs,
        branch_components=["Link"],
        ax=ax,
        geomap=True,
        flow=pd.concat({"Link": n.links.flow.div(linewidth_factor)}),
    )

    colors = [link_color]
    labels = ["Gas network"]
    add_legend_patch(
        ax,
        colors,
        labels,
        patch_kw=dict(edgecolor="k"),
        legend_kw=dict(loc="upper left"),
    )

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
def plot_elec_flow(network, min_energy=2e6, fn=None):  # MWh
    crs = ccrs.EqualEarth()

    n = network.copy()

    assign_location(n)

    # Drop non-electric buses so they don't clutter the plot
    n.buses.drop(n.buses.index[n.buses.carrier != "AC"], inplace=True)

    n.mremove("Link", n.links.index[n.links.carrier != "DC"])

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    link_flow = n.snapshot_weightings.generators @ n.links_t.p0
    line_flow = n.snapshot_weightings.generators @ n.lines_t.p0

    link_flow = link_flow.where(link_flow.abs() > min_energy)
    line_flow = line_flow.where(line_flow.abs() > min_energy)

    n.plot(
        ax=ax,
        geomap=True,
        bus_sizes=0,
        flow=pd.concat(
            {"Link": link_flow.div(4e5), "Line": line_flow.div(4e5)}, axis=0
        ),
        branch_components=["Line", "Link"],
    )

    colors = ["rosybrown", "darkseagreen"]
    labels = ["HVAC lines", "HVDC links"]
    add_legend_patch(
        ax,
        colors,
        labels,
        patch_kw=dict(edgecolor="k"),
        legend_kw=dict(loc="upper left"),
    )

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
if H2_NETWORK:
    io = get_import_export_balance(n)
    plot_h2_flow(
        n,
        nodes,
        io["hydrogen"],
        lim=200,
        min_energy=0,
        fn=f"{OUTPUT_SCENARIO}/h2-flow-map.pdf",
    )
    plot_h2_flow(
        n,
        nodes,
        io["hydrogen"],
        lim=200,
        min_energy=2e6,
        fn=f"{OUTPUT_SCENARIO}/h2-flow-map-backbone.pdf",
    )

In [ ]:
if GAS_NETWORK:
    plot_gas_flow(
        n, min_energy=0, max_energy=100e6, fn=f"{OUTPUT_SCENARIO}/gas-flow-map.pdf"
    )
    plot_gas_flow(
        n,
        min_energy=10e6,
        max_energy=100e6,
        fn=f"{OUTPUT_SCENARIO}/gas-flow-map-backbone.pdf",
    )

In [ ]:
plot_elec_flow(n, min_energy=0, fn=f"{OUTPUT_SCENARIO}/elec-flow-map.pdf")
plot_elec_flow(n, min_energy=10e6, fn=f"{OUTPUT_SCENARIO}/elec-flow-map-backbone.pdf")

## Maps: Curtailment and Line Loading

In [ ]:
def congestion_frequency(n, c):
    pnl = n.pnl(c)
    return n.snapshot_weightings.generators @ (
        pnl.mu_upper.applymap(lambda x: x > 1) | pnl.mu_lower.applymap(lambda x: x > 1)
    )

In [ ]:
def congestion_rent(n, c):
    df = n.df(c)
    pnl = n.pnl(c)

    lmp = n.buses_t.marginal_price
    lmp0 = lmp.reindex(columns=df.bus0).T.reset_index(drop=True)
    lmp1 = lmp.reindex(columns=df.bus1).T.reset_index(drop=True)
    lmp_diff = (lmp0 - lmp1).abs().T
    lmp_diff.columns = df.index

    congestion_rent = (
        lmp_diff * pnl.p0.abs().multiply(n.snapshot_weightings.generators, axis=0) / 1e6
    )

    return congestion_rent.sum()

In [ ]:
def curtailment(n, carriers=None):
    avail = n.generators_t.p_max_pu * n.generators.p_nom_opt
    dispatch = n.generators_t.p

    curtail = n.snapshot_weightings.generators @ (avail - dispatch)

    curtail = (
        curtail.groupby([n.generators.bus.map(n.buses.location), n.generators.carrier])
        .sum()
        .unstack()
    )

    if carriers is not None:
        curtail = curtail[carriers]

    return curtail.drop("EU", axis=0).stack().div(1e6)

In [ ]:
def plot_congestion_rent(network, fn=None):
    n = network.copy()

    carriers = {
        "offwind-ac": "Offshore Wind (AC)",
        "offwind-dc": "Offshore Wind (DC)",
        "onwind": "Onshore Wind",
        "ror": "Run of River",
        "solar": "Utility-scale Solar PV",
        "solar rooftop": "Rooftop Solar PV",
    }

    curtail = curtailment(n, carriers.keys())

    n.buses.drop(n.buses.index[n.buses.carrier != "AC"], inplace=True)
    n.mremove("Link", n.links.index[n.links.carrier != "DC"])

    congestion_rent_link = congestion_rent(n, "Link")
    congestion_rent_line = congestion_rent(n, "Line")

    cmap = plt.cm.OrRd
    norm = mcolors.LogNorm(vmin=10, vmax=2000)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    crs = ccrs.EqualEarth()

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    line_colors = pd.Series(
        list(map(mcolors.to_hex, cmap(norm(congestion_rent_line)))),
        index=congestion_rent_line.index,
    )

    link_colors = pd.Series(
        list(map(mcolors.to_hex, cmap(norm(congestion_rent_link)))),
        index=congestion_rent_link.index,
    )

    bus_size_factor = 50
    line_width_factor = 7e3

    n.plot(
        geomap=True,
        ax=ax,
        bus_sizes=curtail / bus_size_factor,
        bus_colors=tech_colors,
        line_colors=line_colors,
        link_colors=link_colors,
        line_widths=n.lines.s_nom_opt / line_width_factor,
        link_widths=n.links.p_nom_opt / line_width_factor,
        boundaries=BOUNDARIES,
    )

    cbar = plt.colorbar(
        sm,
        orientation="vertical",
        shrink=0.7,
        ax=ax,
        label="Congestion Rent [Million €]",
        extend="max",
    )

    sizes = [5, 10]
    labels = [f"{s} TWh" for s in sizes]
    add_legend_circles(
        ax,
        sizes,
        labels,
        scale=bus_size_factor,
        srid=n.srid,
        legend_kw=dict(title="Curtailment", loc="upper left"),
        patch_kw=dict(facecolor="lightgrey", edgecolor="k"),
    )

    sizes = [10, 20]
    labels = [f"{s} GW" for s in sizes]
    add_legend_lines(
        ax,
        sizes,
        labels,
        scale=line_width_factor / 1e3,
        legend_kw=dict(title="Line Capacity", bbox_to_anchor=(0.5, 1)),
        patch_kw=dict(color="lightgrey"),
    )

    colors = [tech_colors[c] for c in carriers.keys()]
    labels = carriers.values()
    add_legend_patch(
        ax,
        colors,
        labels,
        patch_kw=dict(edgecolor="k"),
        legend_kw=dict(bbox_to_anchor=(1.25, 1.15), ncol=3),
    )

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
def plot_line_loading(network, fn=None):
    line_width_factor = 4e3

    n = network.copy()

    n.mremove("Bus", n.buses.index[n.buses.carrier != "AC"])
    n.mremove("Link", n.links.index[n.links.carrier != "DC"])

    line_loading = (
        n.lines_t.p0.abs().mean() / (n.lines.s_nom_opt * n.lines.s_max_pu) * 100
    )

    link_loading = (
        n.links_t.p0.abs().mean() / (n.links.p_nom_opt * n.links.p_max_pu) * 100
    )

    crs = ccrs.EqualEarth()

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    cmap = plt.cm.OrRd
    norm = mcolors.Normalize(vmin=0, vmax=100)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    line_colors = pd.Series(
        list(map(mcolors.to_hex, cmap(norm(line_loading)))), index=line_loading.index
    )

    link_colors = pd.Series(
        list(map(mcolors.to_hex, cmap(norm(link_loading)))), index=link_loading.index
    )

    n.plot(
        geomap=True,
        ax=ax,
        bus_sizes=0.005,
        bus_colors="k",
        line_colors=line_colors,
        line_widths=n.lines.s_nom_opt / line_width_factor,
        link_colors=link_colors,
        link_widths=n.links.p_nom_opt / line_width_factor,
    )

    cbar = plt.colorbar(
        sm,
        orientation="vertical",
        shrink=0.7,
        ax=ax,
        label="Average Loading / N-1 Compliant Rating [%]",
    )

    sizes = [10, 20]
    labels = [f"{s} GW" for s in sizes]
    add_legend_lines(
        ax,
        sizes,
        labels,
        scale=line_width_factor / 1e3,
        legend_kw=dict(title="Line Capacity", bbox_to_anchor=(1.03, 0.62)),
        patch_kw=dict(color="lightgrey"),
    )

    axins = ax.inset_axes([0.05, 0.8, 0.3, 0.2])
    curve = line_loading.sort_values().reset_index(drop=True)
    curve.index = [c / curve.size * 100 for c in curve.index]
    curve.plot(
        ax=axins,
        ylim=(-5, 105),
        yticks=[0, 25, 50, 75, 100],
        c="firebrick",
        linewidth=1.5,
    )
    axins.annotate("loading [%]", (3, 83), color="darkgrey", fontsize=9)
    axins.annotate("lines [%]", (55, 7), color="darkgrey", fontsize=9)
    axins.grid(True)

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
def plot_pipeline_loading(network, min_capacity=0, fn=None):
    line_width_factor = 6e3

    n = network.copy()

    n.mremove("Bus", n.buses.index[n.buses.carrier != "AC"])
    n.mremove("Link", n.links.index[~n.links.carrier.str.contains("H2 pipeline")])

    n.mremove("Link", n.links.index[n.links.p_nom_opt < min_capacity])
    n.links.bus0 = n.links.bus0.str.replace(" H2", "")
    n.links.bus1 = n.links.bus1.str.replace(" H2", "")

    link_loading = n.links_t.p0.abs().mean() / n.links.p_nom_opt * 100

    crs = ccrs.EqualEarth()

    fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

    cmap = plt.cm.OrRd
    norm = mcolors.Normalize(vmin=0, vmax=100)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    link_colors = pd.Series(
        list(map(mcolors.to_hex, cmap(norm(link_loading)))), index=link_loading.index
    )

    n.plot(
        geomap=True,
        ax=ax,
        bus_sizes=0.005,
        bus_colors="k",
        link_colors=link_colors,
        link_widths=n.links.p_nom_opt / line_width_factor,
        branch_components=["Link"],
    )

    sizes = [10, 20]
    labels = [f"{s} GW" for s in sizes]
    add_legend_lines(
        ax,
        sizes,
        labels,
        scale=line_width_factor / 1e3,
        legend_kw=dict(title="Pipe Capacity", bbox_to_anchor=(1.03, 0.62)),
        patch_kw=dict(color="lightgrey"),
    )

    axins = ax.inset_axes([0.05, 0.8, 0.3, 0.2])
    curve = link_loading.sort_values().reset_index(drop=True)
    curve.index = [c / curve.size * 100 for c in curve.index]
    curve.plot(
        ax=axins,
        ylim=(-5, 105),
        yticks=[0, 25, 50, 75, 100],
        c="firebrick",
        linewidth=1.5,
    )
    axins.annotate("loading [%]", (3, 83), color="darkgrey", fontsize=9)
    axins.annotate("pipes [%]", (55, 7), color="darkgrey", fontsize=9)
    axins.grid(True)

    cbar = plt.colorbar(
        sm,
        orientation="vertical",
        shrink=0.7,
        ax=ax,
        label="Average Loading / Pipe Capacity [%]",
    )

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
plot_congestion_rent(n, fn=f"{OUTPUT_SCENARIO}/map_congestion.pdf")

In [ ]:
plot_line_loading(n, fn=f"{OUTPUT_SCENARIO}/map_avg_lineloading.pdf")

In [ ]:
if H2_NETWORK:
    plot_pipeline_loading(
        n, min_capacity=1000, fn=f"{OUTPUT_SCENARIO}/map_avg_pipelineloading.pdf"
    )

## Stats: TWkm

In [ ]:
def calculate_twkm(n, selection=None, decimals=1, which="optimal"):
    if selection is None:
        selection = [
            "H2 pipeline",
            "H2 pipeline retrofitted",
            "gas pipeline",
            "gas pipeline new",
            "DC",
        ]

    twkm = n.links.loc[n.links.carrier.isin(selection)]

    if which == "optimal":
        link_request = "p_nom_opt"
        line_request = "s_nom_opt"
    elif which == "added":
        link_request = "(p_nom_opt - p_nom)"
        line_request = "(s_nom_opt - s_nom)"
    elif which == "existing":
        link_request = "p_nom"
        line_request = "s_nom"

    twkm = twkm.eval(f"length*{link_request}").groupby(twkm.carrier).sum() / 1e6  # TWkm
    twkm["AC"] = n.lines.eval(f"length*{line_request}").sum() / 1e6  # TWkm

    twkm.index.name = None

    return twkm.round(decimals)

In [ ]:
twkm = calculate_twkm(n, which="optimal")

In [ ]:
twkm

In [ ]:
pd.DataFrame(
    {key: calculate_twkm(n, which=key) for key in ["optimal", "added", "existing"]}
).stack().to_csv(f"{OUTPUT_SCENARIO}/twkm.csv")

## Stats: Energy Moved

In [ ]:
def calculate_energy_moved(n, selection=None, decimals=2):
    if selection is None:
        selection = [
            "DC",
            "H2 pipeline",
            "H2 pipeline retrofitted",
            "gas pipeline",
            "gas pipeline_new",
        ]

    transport = n.links.query("carrier in @selection")

    df = (
        (n.snapshot_weightings.generators @ n.links_t.p0[transport.index].abs())
        .mul(transport.length)
        .groupby(transport.carrier)
        .sum()
    )

    df["AC"] = (
        (n.snapshot_weightings.generators @ n.lines_t.p0.abs())
        .mul(n.lines.length)
        .sum()
    )

    return df.div(1e12).round(decimals)  # EWhkm

In [ ]:
calculate_energy_moved(n).to_csv(f"{OUTPUT_SCENARIO}/ewhkm.csv")

## Maps: Nodal Prices

In [ ]:
def get_nodal_prices(n):
    return (
        n.buses_t.marginal_price.mean()
        .groupby([n.buses.location, n.buses.carrier])
        .first()
        .unstack()
    )

In [ ]:
def plot_nodal_prices(df, geodf, carrier, label, vmin, vmax, cmap="Blues", fn=True):
    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=vmin,
        vmax=vmax,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/nodal-prices-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
df = get_nodal_prices(n)

In [ ]:
carrier = "AC"
plot_nodal_prices(df, nodes, carrier, "mean electricity price [€/MWh]", 50, 120)

In [ ]:
carrier = "H2"
plot_nodal_prices(df, nodes, carrier, "mean hydrogen price [€/MWh]", 80, 140, "RdPu")

In [ ]:
carrier = "residential urban decentral heat"
plot_nodal_prices(
    df,
    nodes,
    carrier,
    "mean heat price [€/MWh]",
    0,
    80,
    "OrRd",
)

In [ ]:
carrier = "urban central heat"
plot_nodal_prices(df, nodes, carrier, "mean heat price [€/MWh]", 0, 80, "OrRd")

In [ ]:
carrier = "residential rural heat"
plot_nodal_prices(df, nodes, carrier, "mean heat price [€/MWh]", 0, 80, "OrRd")

In [ ]:
if GAS_NETWORK:
    carrier = "gas"
    plot_nodal_prices(df, nodes, carrier, "mean gas price [€/MWh]", 0, 30, "OrRd")

## Maps: LCOE

In [ ]:
def calculate_lcoe(n, carrier, p_nom_threshold=1000):
    gen_i = n.generators.loc[n.generators.carrier == carrier].index
    gen = n.snapshot_weightings.generators @ n.generators_t.p.loc[:, gen_i]

    oc = gen * n.generators.loc[gen_i, "marginal_cost"]
    ic = n.generators.eval("capital_cost * p_nom_opt").loc[gen_i]

    lcoe = (ic + oc) / gen

    lcoe = lcoe.where(n.generators.p_nom_opt > p_nom_threshold).dropna()

    lcoe.index = lcoe.index.map(n.generators.bus.map(n.buses.location))

    return lcoe

In [ ]:
def plot_lcoe(n, geodf, carrier, label, vmin, vmax, cmap="Blues", fn=True):
    lcoe = calculate_lcoe(n, carrier)

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=lcoe.reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=vmin,
        vmax=vmax,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    plt.xlim(-1e6, 2.6e6)
    plt.ylim(4.3e6, 7.8e6)

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/lcoe-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")

    plt.close()

In [ ]:
plot_lcoe(n, nodes, "onwind", "onshore wind LCOE [€/MWh]", 0, 100)

In [ ]:
plot_lcoe(n, offnodes, "offwind-dc", "offshore wind (DC) LCOE [€/MWh]", 0, 100)

In [ ]:
plot_lcoe(n, offnodes, "offwind-ac", "offshore wind (AC) LCOE [€/MWh]", 0, 100)

In [ ]:
plot_lcoe(n, nodes, "solar rooftop", "rooftop solar LCOE [€/MWh]", 0, 100, "Oranges")

In [ ]:
plot_lcoe(n, nodes, "solar", "utility solar LCOE [€/MWh]", 0, 100, "Oranges")

## Maps: Market Values

In [ ]:
def market_values(n, carrier="onwind"):
    gen = n.generators_t.p.loc[:, n.generators.carrier == carrier]
    gen.columns = gen.columns.map(n.generators.bus)

    lmp = n.buses_t.marginal_price.loc[:, gen.columns]

    mv = (gen * lmp).sum() / gen.sum()

    mv.index = mv.index.map(n.buses.location)

    return mv

In [ ]:
def plot_market_values(n, geodf, carrier, label, cmap="Blues", fn=True):
    mv = market_values(n, carrier)

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=mv.reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        # vmin=vmin,
        # vmax=vmax,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/market-values-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
plot_market_values(n, nodes, "solar", "utility-solar market value [€/MWh]", "Oranges")

In [ ]:
plot_market_values(
    n, nodes, "solar rooftop", "rooftop solar market value [€/MWh]", "Oranges"
)

In [ ]:
plot_market_values(n, nodes, "onwind", "onshore wind market value [€/MWh]", "Blues")

In [ ]:
plot_market_values(
    n, offnodes, "offwind-dc", "offshore wind (DC) market value [€/MWh]", "Blues"
)

In [ ]:
plot_market_values(
    n, offnodes, "offwind-ac", "offshore wind (AC) market value [€/MWh]", "Blues"
)

In [ ]:
plot_market_values(n, nodes, "ror", "run-of-river market value [€/MWh]", "Greens")

## Potential Used

In [ ]:
def potential_used(n):
    return (
        n.generators.eval("p_nom_opt/p_nom_max*100")
        .groupby([n.generators.bus.map(n.buses.location), n.generators.carrier])
        .sum()
        .unstack()
        .drop("EU")
    )

In [ ]:
def plot_potential_used(df, geodf, carrier, cmap="Blues", fn=True):
    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        # transform=ccrs.PlateCarree(),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=100,
        vmin=0,
        legend_kwds={
            "label": "share of technical potential used [%]",
            "shrink": 0.7,
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/potential-used-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
df = potential_used(n)

In [ ]:
plot_potential_used(df, offnodes, "offwind-ac")

In [ ]:
plot_potential_used(df, offnodes, "offwind-dc")

In [ ]:
plot_potential_used(df, nodes, "onwind")

In [ ]:
plot_potential_used(df, nodes, "solar", cmap="Oranges")

In [ ]:
plot_potential_used(df, nodes, "solar rooftop", cmap="Oranges")

## Import and Export

In [ ]:
def plot_import_export(
    df,
    geodf,
    carrier,
    cmap="PiYG",
    unit="TWh",
    fn=True,
    lim=None,
    lorenz=False,
    dfs={},
):
    crs = ccrs.EqualEarth()
    geodf = geodf.to_crs(crs.proj4_init)

    if lim is None:
        lim = df[carrier].abs().max()

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": crs})

    geodf.plot(
        ax=ax,
        column=-df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=-lim,
        vmax=lim,
        legend_kwds={
            "label": f"{carrier} balance [{unit}]",
            "shrink": 0.7,
            "extend": "both",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if lorenz:
        axins = ax.inset_axes([0.05, 0.77, 0.27, 0.23])
        carriers = ["AC", "H2", "gas", "oil"]  # if GAS_NETWORK else ["AC", "H2"]
        plot_lorenz_inset(axins, n, carriers, dfs=dfs)

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/import-export-{carrier}-{lim}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
def plot_lorenz_inset(ax, n, carriers, sortby="supply/demand", dfs={}):
    for c in set(carriers).difference(dfs.keys()):
        dfs[c] = None

    linspace = np.linspace(0, 100, 100)
    ax.plot(linspace, linspace, c="darkgrey", linestyle=":", linewidth=1)

    nice_names = {
        "AC": "electricity",
        "H2": "hydrogen",
        "gas": "methane",
        "oil": "oil",
    }

    linestyles = {
        "AC": "solid",
        "H2": "dashed",
        "oil": "dashdot",
        "gas": "dotted",
    }

    for carrier in carriers:
        df = cumulative_share(n, carrier, sortby, df=dfs[carrier])
        ax.plot(
            df.demand,
            df.supply,
            label=nice_names[carrier],
            color=tech_colors[nice_names[carrier]],
            linestyle=linestyles[carrier],
            linewidth=1.5,
        )

    ax.set_xlabel(f"Demand [%]", fontsize=9, backgroundcolor="white", labelpad=-7)
    ax.set_ylabel(f"Supply [%]", fontsize=9, backgroundcolor="white", labelpad=-10)
    ax.set_ylim(-1, 101)
    ax.set_xlim(-1, 101)
    ax.tick_params(labelsize=8)
    ax.legend(fontsize=9, loc=(1.01, 0.3), frameon=False)
    for i in ["top", "right", "left", "bottom"]:
        ax.spines[i].set_visible(False)

In [ ]:
io = get_import_export_balance(n, with_methanol=False)

In [ ]:
dfs = {"oil": get_import_export_oil(with_methanol=True)}
if not GAS_NETWORK:
    dfs["gas"] = get_import_export_gas()

In [ ]:
plot_import_export(io, nodes, "electricity", cmap="PRGn", lim=200)

In [ ]:
plot_import_export(io, nodes, "electricity", cmap="PRGn", lim=100)

In [ ]:
plot_import_export(io, nodes, "hydrogen", cmap="PRGn", lim=200)

In [ ]:
plot_import_export(io, nodes, "gas", cmap="PRGn", lim=200)
plot_import_export(io, nodes, "gas", cmap="PRGn", lim=40)

In [ ]:
plot_import_export(io, nodes, "total", cmap="PRGn", lim=200, lorenz=True, dfs=dfs)

## Capacities Built

In [ ]:
def plot_capacity_built(df, geodf, carrier, cmap="Blues", unit="GW", fn=None):
    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=0,
        legend_kwds={
            "label": f"{carrier} capacity [{unit}]",
            "shrink": 0.7,
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(
            f"{OUTPUT_SCENARIO}/capacities-built-{carrier}.pdf", bbox_inches="tight"
        )
        plt.close()

In [ ]:
def calculate_p_caps(n):
    p_caps = pd.concat(
        [
            n.generators.groupby([n.generators.bus.map(n.buses.location), "carrier"])
            .p_nom_opt.sum()
            .unstack()
            .drop(["oil", "gas"], axis=1),
            n.links.groupby([n.links.bus0.map(n.buses.location), "carrier"])
            .p_nom_opt.sum()
            .unstack()
            .drop(["DC", "H2 pipeline"], axis=1, errors="ignore"),
            n.storage_units.groupby(
                [n.storage_units.bus.map(n.buses.location), "carrier"]
            )
            .p_nom_opt.sum()
            .unstack(),
        ],
        axis=1,
    ).div(
        1e3
    )  # GW

    return p_caps.loc[:, p_caps.count() > 1]

In [ ]:
p_caps = calculate_p_caps(n)

In [ ]:
for i in p_caps.columns:
    if "off" in i:
        gdf = offnodes
    else:
        gdf = nodes

    if "wind" in i:
        cmap = "Blues"
    elif "heat" in i or "water tank" in i:
        cmap = "Reds"
    elif "solar" in i:
        cmap = "Oranges"
    elif i in ["hydro", "PHS", "ror"]:
        cmap = "GnBu"
    elif "H2" in i or i in ["Fischer-Tropsch", "Sabatier", "methanolisation"]:
        cmap = "RdPu"
    elif "battery" in i or "distribution" in i or "BEV" in i or "V2G" in i:
        cmap = "Greens"

    if p_caps[i].max() > 1:
        plot_capacity_built(p_caps, gdf, i, cmap);

In [ ]:
def calculate_e_caps(n):
    e_caps = (
        n.stores.groupby([n.stores.bus.map(n.buses.location), "carrier"])
        .e_nom_opt.sum()
        .unstack()
        .div(1e3)
    )  # GWh
    return e_caps.loc[:, e_caps.count() > 1]

In [ ]:
e_caps = calculate_e_caps(n)

In [ ]:
for i in e_caps.columns:
    if "water tank" in i:
        cmap = "Reds"
    elif "solar" in i:
        cmap = "Oranges"
    elif "H2" in i or "gas" in i:
        cmap = "RdPu"
    elif "battery" in i or "Li ion":
        cmap = "Greens"

    if e_caps[i].max() > 1:
        plot_capacity_built(e_caps, nodes, i, cmap, unit="GWh");

## Utilisation Factor Time Series

In [ ]:
def aggregate_techs(tech):
    if "solar thermal" in tech:
        return "solar thermal"
    elif "solar" in tech:
        return "solar PV"
    elif "offwind" in tech:
        return "offshore wind"
    elif "onwind" in tech:
        return "onshore wind"
    elif "ror" in tech:
        return "run of river"
    elif "ground heat pump" in tech:
        return "ground-sourced heat pump"
    elif "air heat pump" in tech:
        return "air-sourced heat pump"
    elif "water tank" in tech:
        return "thermal energy storage"
    elif tech == "H2":
        return "hydrogen storage"
    elif tech == "Li ion":
        return "electric vehicle batteries"
    elif "gas boiler" in tech:
        return "gas boiler"
    elif "resistive heater" in tech:
        return "resistive heater"
    elif "CHP" in tech:
        return "CHP"
    else:
        return tech

In [ ]:
def unstack_day_hour(cfc):
    df = cfc.groupby(cfc.index.hour).agg(list)

    columns = pd.date_range(cfc.index[0], cfc.index[-1], freq="D").strftime("%-d %b")

    return pd.DataFrame(df.tolist(), index=df.index, columns=columns)

In [ ]:
def plot_cf_heatmap(
    df, vmin=0, vmax=80, cmap="Greens", label="capacity factors [%]", fn=True
):
    fig, ax = plt.subplots(figsize=(6, 2.5))
    sns.heatmap(
        df,
        cmap=cmap,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        cbar_kws=dict(label=label, extend="max"),
    )
    plt.ylabel("hour of the day")
    plt.xlabel("day of the year")
    plt.yticks(rotation=0)
    plt.title(carrier, fontsize=12)
    plt.tight_layout()
    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/cf-ts-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
cf = n.buses_t.marginal_price.groupby(n.buses.carrier, axis=1).mean()

In [ ]:
# for carrier in cf.columns:
for carrier in ["AC", "H2"]:
    cfc = cf[carrier]
    df = unstack_day_hour(cfc)
    vmax = np.ceil(cfc.quantile(0.99) / 10) * 10
    if carrier in ["AC", "H2"]:
        vmax = 300
    plot_cf_heatmap(
        df,
        cmap="Spectral_r",
        label="Nodal Price [EUR/MWh]",
        vmax=vmax,
        fn=f"{OUTPUT_SCENARIO}/price-ts-{carrier}.pdf",
    )

In [ ]:
cf = n.links_t.p_max_pu.groupby(n.links.carrier, axis=1).mean() * 100
cf = cf.groupby(cf.columns.map(aggregate_techs), axis=1).mean()

In [ ]:
for carrier in cf.columns:
    cfc = cf[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(df, cmap="viridis", vmin=40, vmax=100)

In [ ]:
cf = (
    n.stores_t.e.groupby(n.stores.carrier, axis=1).sum()
    / n.stores.e_nom_opt.groupby(n.stores.carrier).sum()
    * 100
)
cf = cf.groupby(cf.columns.map(aggregate_techs), axis=1).mean()

In [ ]:
for carrier in cf.columns:
    cfc = cf[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(df, cmap="Purples", vmin=0, vmax=100, label="SOC [%]")

In [ ]:
cf = (
    n.links_t.p0.groupby(n.links.carrier, axis=1).sum()
    / n.links.p_nom_opt.groupby(n.links.carrier).sum()
    * 100
)
cf = cf.groupby(cf.columns.map(aggregate_techs), axis=1).mean()

In [ ]:
bidirectional = (cf < 0).any()

In [ ]:
cfb = cf.loc[:, ~bidirectional]
for carrier in cfb.columns:
    cfc = cfb[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(df, cmap="Reds", vmax=100)

In [ ]:
cfb = cf.loc[:, bidirectional]
for carrier in cfb.columns:
    cfc = cfb[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(df, cmap="RdBu", vmin=-100, vmax=100)

In [ ]:
cf = n.generators_t.p_max_pu.groupby(n.generators.carrier, axis=1).mean() * 100
cf = cf.groupby(cf.columns.map(aggregate_techs), axis=1).mean()

In [ ]:
for carrier in cf.columns:
    cfc = cf[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(df, cmap="Blues", fn=f"{OUTPUT}/cf-raw-ts-{carrier}.pdf")

In [ ]:
cf = n.links_t.efficiency.groupby(n.links.carrier, axis=1).mean()
cf = cf.groupby(cf.columns.map(aggregate_techs), axis=1).mean()

In [ ]:
for carrier in cf.columns:
    cfc = cf[carrier]
    df = unstack_day_hour(cfc)
    plot_cf_heatmap(
        df,
        vmin=1,
        vmax=4,
        cmap="Greens",
        label="COP [-]",
        fn=f"{OUTPUT}/cop-ts-{carrier}.pdf",
    )

## Maps: Utilisation Factors

In [ ]:
grouper = [n.links.carrier, n.links.bus0.map(n.buses.location)]

In [ ]:
c = n.links.p_nom_opt.groupby(grouper).sum()
c = c.where(c > 250)

In [ ]:
cf = n.links_t.p0.groupby(grouper, axis=1).sum() / c * 100

In [ ]:
df = cf.mean().unstack(0)

In [ ]:
def plot_utilisation_factors(
    df, geodf, carrier, label, vmin=0, vmax=100, cmap="Blues", fn=True
):
    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=vmin,
        vmax=vmax,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    bounds = geodf.total_bounds
    plt.xlim([bounds[0], bounds[2]])
    plt.ylim([bounds[1], bounds[3]])

    ax.set_frame_on(False)
    ax.set_facecolor("white")

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/utilisation-factors-{carrier}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
for tech in df.filter(
    regex="(Sabatier|electricity distribution grid|H2 Electrolysis|H2 Fuel Cell|Fischer-Tropsch|methanolisation)"
).columns:
    plot_utilisation_factors(
        df, nodes, tech, f"{tech} Utilisation Factor [%]", cmap="RdPu"
    )

## Balance Time Series

In [ ]:
def rename_techs_tyndp(tech):
    tech = rename_techs(tech)
    # if "heat pump" in tech or "resistive heater" in tech:
    #    return "power-to-heat"
    # elif tech in ["H2 Electrolysis", "methanation", "helmeth", "H2 liquefaction"]:
    #    return "power-to-gas"
    if tech == "H2":
        return "H2 storage"
    # elif tech in ["OCGT", "CHP", "gas boiler", "H2 Fuel Cell"]:
    #    return "gas-to-power/heat"
    # elif "solar" in tech:
    #    return "solar"
    elif tech == "Fischer-Tropsch":
        return "power-to-liquid"
    elif "offshore wind" in tech:
        return "offshore wind"
    #    if "heat pump" in tech:
    #        return "heat pump"
    elif tech == "gas":
        return "fossil gas"
    # elif "CC" in tech or "sequestration" in tech:
    #    return "CCS"
    elif tech in ["industry electricity", "agriculture electricity"]:
        return "industry electricity"
    elif "oil emissions" in tech:
        return "oil emissions"
    else:
        return tech

In [ ]:
preferred_order = pd.Index(
    [
        "H2 storage",
        "hydrogen storage",
        "battery storage",
        "BEV charger",
        "V2G",
        "hot water storage",
        "co2",
        "hydroelectricity",
        "hydro reservoir",
        "pumped hydro storage",
        "run of river",
        "transmission lines",
        "electricity distribution grid",
        "solid biomass",
        "biogas",
        "onshore wind",
        "offshore wind",
        "offshore wind (AC)",
        "offshore wind (DC)",
        "solar PV",
        "solar thermal",
        "solar rooftop",
        "solar",
        "building retrofitting" "ground heat pump",
        "air heat pump",
        "heat pump",
        "resistive heater",
        "power-to-heat",
        "gas-to-power/heat",
        "CHP",
        "OCGT",
        "gas boiler",
        "gas",
        "natural gas",
        "helmeth",
        "methanation",
        "power-to-gas",
        "power-to-H2",
        "H2 pipeline",
        "H2 pipeline retrofitted",
        "gas pipeline",
        "gas pipeline new",
        "H2 liquefaction",
        "power-to-liquid",
        "CO2 sequestration",
        "CCS",
    ]
)

In [ ]:
def datetime_xticks(
    df,
    ax,
    number_timestamps=10,
    minor_attr="day",
    minor_fmt="%d",
    major_attr="month",
    major_fmt="%b",
):
    # https://stackoverflow.com/questions/30133280/pandas-bar-plot-changes-date-format

    # Create list of monthly timestamps by selecting the first weekly timestamp of each
    # month (in this example, the first Sunday of each month)
    daily_timestamps = [
        timestamp
        for idx, timestamp in enumerate(df.index)
        if (getattr(timestamp, minor_attr) != getattr(df.index[idx - 1], minor_attr))
        | (idx == 0)
    ]

    # Automatically select appropriate number of timestamps so that x-axis does
    # not get overcrowded with tick labels
    step = 1
    while len(daily_timestamps[::step]) > number_timestamps:
        step += 1
    timestamps = daily_timestamps[::step]

    # Create tick labels from timestamps
    labels = [
        ts.strftime(f"{minor_fmt}\n{major_fmt}")
        if (getattr(ts, major_attr) != getattr(timestamps[idx - 1], major_attr))
        | (idx == 0)  # | (idx == len(timestamps)-1)
        else ts.strftime(minor_fmt)
        for idx, ts in enumerate(timestamps)
    ]

    # Set major ticks and labels
    ax.set_xticks([df.index.get_loc(ts) for ts in timestamps])
    ax.set_xticklabels(labels)

    # Set minor ticks without labels
    ax.set_xticks([df.index.get_loc(ts) for ts in daily_timestamps], minor=True)

    # Rotate and center labels
    ax.figure.autofmt_xdate(rotation=0, ha="center")

In [ ]:
def plot_balance_timeseries(
    n, carrier, time, ylims=None, resample="", balance=None, final_energy=False, fn=True
):
    if balance is None:
        balance = nodal_balance(n, carrier, time=time, energy=False)

    df = balance.groupby(["carrier", "snapshot"]).sum().unstack(0).div(1e3)

    df = df.groupby(df.columns.map(rename_techs_tyndp), axis=1).sum()

    if resample:
        df = df.resample(resample).mean()

    df = df.loc[:, ~df.columns.isin(["H2 pipeline", "transmission lines"])]

    order = preferred_order.intersection(df.columns).append(
        df.columns.difference(preferred_order)
    )
    df = df.loc[:, order]

    colors = df.columns.map(tech_colors)

    fig, ax = plt.subplots(figsize=(10, 4))

    dft = df.loc[time]

    pos = dft.where(dft > 0)
    neg = dft.where(dft < 0)

    if final_energy:
        e = -dft[["electricity", "industry electricity"]].sum(axis=1)
        e.plot(ax=ax, c="k", linewidth=1, use_index=False, drawstyle="steps-mid")

    kwargs = dict(ax=ax, linewidth=0, color=colors, width=1.0)

    pos.plot.bar(**kwargs, stacked=True)
    neg.plot.bar(**kwargs, stacked=True)

    handles, labels = ax.get_legend_handles_labels()

    half = int(len(handles) / 2)
    ax.legend(
        handles=handles[:half],
        labels=labels[:half],
        bbox_to_anchor=(1, 1.05),
    )

    if ylims is None:
        ylim = np.ceil(max(-neg.sum(axis=1).min(), pos.sum(axis=1).max()) / 100) * 100
    else:
        ylim = ylims[carrier] if isinstance(ylims, dict) else ylims

    ax.set_ylim([-ylim, ylim])
    ax.set_xlabel("")
    unit = "kt/h" if "co2" in carrier else "GW"
    ax.set_ylabel(f"{carrier} balance [{unit}]")

    if "W" in resample or "D" in resample:
        datetime_xticks(
            dft, ax, number_timestamps=12, minor_attr="month", minor_fmt="%d"
        )
    else:
        datetime_xticks(dft, ax)

    ax.axhline(0, color="grey", linewidth=0.5)

    if fn:
        if not isinstance(fn, str):
            fn = f"{OUTPUT_SCENARIO}/ts-balance-{carrier}-{resample}-{time}.pdf"
        plt.savefig(fn, bbox_inches="tight")
        plt.close()

In [ ]:
carriers = [
    "AC",
    "co2",
    "co2 stored",
    "gas",
    "H2",
    "residential rural heat",
    "services rural heat",
    "residential urban decentral heat",
    "services urban decentral heat",
    "urban central heat",
    "oil",
    "methanol",
    "low voltage",
]

In [ ]:
ylims = {
    "AC": 2200,
    "co2": 150,
    "co2 stored": 100,
    "gas": 700,
    "H2": 800,
    "residential rural heat": 200,
    "services rural heat": 100,
    "residential urban decentral heat": 200,
    "services urban decentral heat": 100,
    "urban central heat": 600,
    "oil": 200,
    "methanol": 100,
    "low voltage": 1000,
}

In [ ]:
months = pd.date_range("2013-01", "2014-01", freq="M", inclusive="right").format(
    formatter=lambda x: x.strftime("%Y-%m")
)

In [ ]:
months = ["2013-02", "2013-07"]

In [ ]:
def multiprocess(f, variants, nprocesses=2):
    nprocesses = min(mp.cpu_count(), nprocesses)

    with mp.Pool(processes=nprocesses) as pool:
        x = pool.starmap(f, variants)

In [ ]:
variants = [(n, x, y, ylims) for x in carriers for y in months]
multiprocess(plot_balance_timeseries, variants, 2)

In [ ]:
variants = [(n, carrier, "2013", ylims, "D") for carrier in carriers]
multiprocess(plot_balance_timeseries, variants, 4)

In [ ]:
heat_nodes = [
    "residential rural heat",
    "services rural heat",
    "residential urban decentral heat",
    "services urban decentral heat",
    "urban central heat",
]

In [ ]:
ylim = 1200
for month in months:
    balance = nodal_balance(n, heat_nodes, time=month, energy=False)
    plot_balance_timeseries(n, "total heat", month, ylims=ylim, balance=balance)

In [ ]:
balance = nodal_balance(n, heat_nodes, time="2013", energy=False)
plot_balance_timeseries(
    n, "total heat", "2013", resample="D", ylims=1200, balance=balance
)

In [ ]:
elec_nodes = [
    "AC",
    "low voltage",
]
ylim = 2400

In [ ]:
for month in months:
    balance = nodal_balance(n, elec_nodes, time=month, energy=False)
    plot_balance_timeseries(n, "total electricity", month, ylims=ylim, balance=balance)

In [ ]:
balance = nodal_balance(n, elec_nodes, time="2013", energy=False)
plot_balance_timeseries(
    n, "total electricity", "2013", resample="D", ylims=ylim, balance=balance
)

In [ ]:
liquid_hydrocarbons = [
    "oil",
    "methanol",
]
ylim = 300

In [ ]:
for month in months:
    balance = nodal_balance(n, liquid_hydrocarbons, time=month, energy=False)
    plot_balance_timeseries(
        n, "liquid hydrocarbons", month, ylims=ylim, balance=balance
    )

In [ ]:
balance = nodal_balance(n, liquid_hydrocarbons, time="2013", energy=False)
plot_balance_timeseries(
    n, "liquid hydrocarbons", "2013", resample="D", ylims=ylim, balance=balance
)

## Nodal CSV Report

In [ ]:
report = {}
report["curtailment"] = curtailment(n).unstack()
report["prices"] = get_nodal_prices(n).groupby(rename_techs_tyndp, axis=1).mean()
report["lcoe"] = pd.concat(
    {
        c: calculate_lcoe(n, c)
        for c in ["onwind", "offwind-ac", "offwind-dc", "solar rooftop", "solar"]
    },
    axis=1,
)
report["market_value"] = pd.concat(
    {
        c: market_values(n, c)
        for c in ["onwind", "offwind-ac", "offwind-dc", "solar rooftop", "solar", "ror"]
    },
    axis=1,
)
report["io"] = get_import_export_balance(n)
report["capacity"] = calculate_p_caps(n).groupby(rename_techs_tyndp, axis=1).sum()
report["storage"] = calculate_e_caps(n).groupby(rename_techs_tyndp, axis=1).sum()
report["potential_used"] = potential_used(n)

In [ ]:
report = {k: v.round(2) for k, v in report.items()}
pd.concat(report, axis=1).to_csv(f"{OUTPUT_SCENARIO}/report.csv")

## Network Graph Export

In [ ]:
rename_link_attrs = {
    "p_nom": "s_nom",
    "p_nom_opt": "s_nom_opt",
    "p_nom_max": "s_nom_max",
}
selection = ["bus0", "bus1", "length", "s_nom", "s_nom_max", "carrier", "s_nom_opt"]

edges = pd.concat(
    [
        n.lines,
        n.links.loc[n.links.carrier == "DC"].rename(rename_link_attrs, axis=1),
    ],
    axis=0,
)[selection]
edges.length = edges.length.astype(int)
s_ = ["s_nom", "s_nom_max", "s_nom_opt"]
edges[s_] = edges[s_].div(1e3).round(3)

edges.to_csv(f"{OUTPUT_SCENARIO}/edges-electricity.csv")

In [ ]:
edges = n.links.loc[n.links.carrier.str.contains("H2 pipeline")]

if not edges.empty:
    positive_order = edges.bus0 < edges.bus1
    edges_p = edges[positive_order]
    swap_buses = {"bus0": "bus1", "bus1": "bus0"}
    edges_n = edges[~positive_order].rename(columns=swap_buses)
    edges = pd.concat([edges_p, edges_n])

    edges.length = edges.length.astype(int)
    edges.bus0 = edges.bus0.str.replace(" H2", "")
    edges.bus1 = edges.bus1.str.replace(" H2", "")
    edges.p_nom_max = edges.p_nom_max.div(1e3).round(3)
    edges.p_nom_opt = edges.p_nom_opt.div(1e3).round(3)

    edges["max_retro"] = edges.p_nom_max.replace(np.inf, 0)

    edges["p_nom_opt_new"] = edges.loc[edges.carrier == "H2 pipeline", "p_nom_opt"]

    edges["p_nom_opt_retro"] = edges.loc[
        edges.carrier == "H2 pipeline retrofitted", "p_nom_opt"
    ]

    strategy = {
        "length": "mean",
        "max_retro": "sum",
        "p_nom_opt": "sum",
        "p_nom_opt_new": "sum",
        "p_nom_opt_retro": "sum",
    }
    edges = edges.groupby(["bus0", "bus1"], as_index=False).agg(strategy)
    edges.index = edges.bus0 + " - " + edges.bus1

    edges.to_csv(f"{OUTPUT_SCENARIO}/edges-hydrogen.csv")

In [ ]:
n.buses.query("carrier == 'AC'")[["x", "y", "country"]].to_csv(f"{OUTPUT}/buses.csv")

## CSV exports

In [ ]:
from itertools import product

LL = ["1.0", "opt"]
HY = ["", "-noH2network"]
RUN = ["20221227-main", "20221227-import", "20221227-shipping", "20221227-costs"]

In [ ]:
ONW = ["-onwind+p0", "-noH2network-onwind+p0"]
ONW_RUN = ["20221227-main"]

In [ ]:
for ll, hy, run in list(product(LL, HY, RUN)) + list(product(LL, ONW, ONW_RUN)):
    print(ll, hy, run)

    CLUSTERS = 181
    SCENARIO = f"elec_s_181_lv{ll}__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10{hy}_2050"
    OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

    OUTPUT = "../results/graphics-20221227/"
    OUTPUT_SCENARIO = f"{OUTPUT}/{run}/{SCENARIO}/"

    if not os.path.exists(OUTPUT_SCENARIO):
        os.makedirs(OUTPUT_SCENARIO)

    overrides = override_component_attrs(OVERRIDES)
    fn = f"{PATH}/pypsa-eur-sec/results/{run}/postnetworks/{SCENARIO}.nc"
    n = pypsa.Network(fn, override_component_attrs=overrides)

    unique_link_carriers = n.links.carrier.unique()
    GAS_NETWORK = "gas pipeline" in unique_link_carriers
    H2_NETWORK = any("H2 pipeline" in ulc for ulc in unique_link_carriers)

    carriers = [
        "co2",
        "co2 stored",
        "gas",
        "H2",
    ]

    for carrier in carriers:
        print(carrier)
        df = (
            nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
            .unstack()
            .fillna(0.0)
            .div(1e3)
            .round(1)
        )
        df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-{carrier}.csv")

    carrier = ["AC", "low voltage"]
    df = (
        nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
        .unstack()
        .fillna(0.0)
        .div(1e3)
        .round(1)
    )
    df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-total-electricity.csv")

    carrier = ["oil", "methanol"]
    df = (
        nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
        .unstack()
        .fillna(0.0)
        .div(1e3)
        .round(1)
    )
    df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-total-liquid-hydrocarbons.csv")

    carrier = [
        "residential rural heat",
        "services rural heat",
        "residential urban decentral heat",
        "services urban decentral heat",
        "urban central heat",
    ]
    df = (
        nodal_balance(n, carrier, aggregate=["bus", "component"], energy=False)
        .unstack()
        .fillna(0.0)
        .div(1e3)
        .round(1)
    )
    df.to_csv(f"{OUTPUT_SCENARIO}/balance-ts-total-heat.csv")

    report = {}
    report["curtailment"] = curtailment(n).unstack()
    report["prices"] = get_nodal_prices(n).groupby(rename_techs_tyndp, axis=1).mean()
    report["lcoe"] = pd.concat(
        {
            c: calculate_lcoe(n, c)
            for c in ["onwind", "offwind-ac", "offwind-dc", "solar rooftop", "solar"]
        },
        axis=1,
    )
    report["market_value"] = pd.concat(
        {
            c: market_values(n, c)
            for c in [
                "onwind",
                "offwind-ac",
                "offwind-dc",
                "solar rooftop",
                "solar",
                "ror",
            ]
        },
        axis=1,
    )
    report["io"] = get_import_export_balance(n)
    report["capacity"] = calculate_p_caps(n).groupby(rename_techs_tyndp, axis=1).sum()
    report["storage"] = calculate_e_caps(n).groupby(rename_techs_tyndp, axis=1).sum()
    report["potential_used"] = potential_used(n)

    report = {k: v.round(2) for k, v in report.items()}
    pd.concat(report, axis=1).to_csv(f"{OUTPUT_SCENARIO}/report.csv")

    rename_link_attrs = {
        "p_nom": "s_nom",
        "p_nom_opt": "s_nom_opt",
        "p_nom_max": "s_nom_max",
    }
    selection = ["bus0", "bus1", "length", "s_nom", "s_nom_max", "carrier", "s_nom_opt"]

    edges = pd.concat(
        [
            n.lines,
            n.links.loc[n.links.carrier == "DC"].rename(rename_link_attrs, axis=1),
        ],
        axis=0,
    )[selection]
    edges.length = edges.length.astype(int)
    s_ = ["s_nom", "s_nom_max", "s_nom_opt"]
    edges[s_] = edges[s_].div(1e3).round(3)

    edges.to_csv(f"{OUTPUT_SCENARIO}/edges-electricity.csv")

    edges = n.links.loc[n.links.carrier.str.contains("H2 pipeline")]

    if not edges.empty:
        positive_order = edges.bus0 < edges.bus1
        edges_p = edges[positive_order]
        swap_buses = {"bus0": "bus1", "bus1": "bus0"}
        edges_n = edges[~positive_order].rename(columns=swap_buses)
        edges = pd.concat([edges_p, edges_n])

        edges.length = edges.length.astype(int)
        edges.bus0 = edges.bus0.str.replace(" H2", "")
        edges.bus1 = edges.bus1.str.replace(" H2", "")
        edges.p_nom_max = edges.p_nom_max.div(1e3).round(3)
        edges.p_nom_opt = edges.p_nom_opt.div(1e3).round(3)

        edges["max_retro"] = edges.p_nom_max.replace(np.inf, 0)

        edges["p_nom_opt_new"] = edges.loc[edges.carrier == "H2 pipeline", "p_nom_opt"]

        edges["p_nom_opt_retro"] = edges.loc[
            edges.carrier == "H2 pipeline retrofitted", "p_nom_opt"
        ]

        strategy = {
            "length": "mean",
            "max_retro": "sum",
            "p_nom_opt": "sum",
            "p_nom_opt_new": "sum",
            "p_nom_opt_retro": "sum",
        }
        edges = edges.groupby(["bus0", "bus1"], as_index=False).agg(strategy)
        edges.index = edges.bus0 + " - " + edges.bus1

        edges.to_csv(f"{OUTPUT_SCENARIO}/edges-hydrogen.csv")

    n.buses.query("carrier == 'AC'")[["x", "y", "country"]].to_csv(
        f"{OUTPUT}/buses.csv"
    )

## Trade